In [1]:
import pandas as pd

In [2]:
#5. Nachkommastelle eine 8 drauf
SPX_PRICE=pd.read_csv('data/SPX.csv')
spx=SPX_PRICE[["Date", 'Close']]
spx=spx.drop(range(0, 10496))
spx["daily_return"] = spx['Close'].pct_change()
spx["daily_return"] =spx["daily_return"] #+0.00017

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Annahme:
# Dein DataFrame 'spx' enthält mindestens folgende Spalten:
#   Date         – Datum (als datetime oder String)
#   Close        – Schlusskurs
#   daily_return – Tagesrendite als Dezimalzahl (z.B. 0.01 für +1%)
# -------------------------------

# 1) Steuerfreie Berechnungen

# 1.1) 200-Tage-Durchschnitt berechnen
spx['ma_200'] = spx['Close'].rolling(window=200).mean()

# 1.2) Klassischer Indikator: True, wenn Close über dem 200-Tage-Durchschnitt liegt
spx['in_risk_on'] = spx['Close'] > spx['ma_200']

# 1.3) Verzögertes Signal: Am Tag t wird der Zustand des Vortages genutzt.
spx['signal_3x'] = np.where(spx['in_risk_on'].shift(1, fill_value=True), 3, 0)
spx['signal_2x'] = np.where(spx['in_risk_on'].shift(1, fill_value=True), 2, 0)

# 1.4) Variante: Indikator mit 200-Tage × 0.98 (nur für 3x)
spx['in_risk_on_0_9'] = spx['Close'] > (spx['ma_200'] * 0.98)
spx['signal_3x_0_9'] = np.where(spx['in_risk_on_0_9'].shift(1, fill_value=True), 3, 0)

# 1.5) Tagesrenditen (steuerfrei) der Strategien
spx['strategy_return_3x']     = spx['signal_3x'] * spx['daily_return']
spx['strategy_return_2x']     = spx['signal_2x'] * spx['daily_return']
spx['strategy_return_3x_0_9'] = spx['signal_3x_0_9'] * spx['daily_return']

# 1.6) Kumulative Performance (steuerfrei)
spx['strategy_cum_3x']     = (1 + spx['strategy_return_3x']).cumprod()
spx['strategy_cum_2x']     = (1 + spx['strategy_return_2x']).cumprod()
spx['strategy_cum_3x_0_9'] = (1 + spx['strategy_return_3x_0_9']).cumprod()

# 1.7) Buy & Hold-Performance (ohne Hebel)
spx['buy_and_hold'] = (1 + spx['daily_return']).cumprod()

# 1.8) Nicht-tradendes, 3x-gehebeltes Portfolio:
# Hier wird einfach täglich der 3x-Faktor angewandt, ohne Signalwechsel.
spx['strategy_cum_3x_nontrading'] = (1 + 3 * spx['daily_return']).cumprod()

# 1.9) Entferne Zeilen, in denen der 200-Tage-Durchschnitt (ma_200) noch nicht berechnet werden konnte
spx = spx.dropna(subset=['ma_200']).reset_index(drop=True)

# --------------------------------------------------
# 2) Simulation der Steuer-Anpassung
#    (Die Steuer wird beim Übergang vom Leveraged-Modus in risk-off berechnet:
#     Es wird 20% des Gewinns seit dem letzten Wechsel (Baseline) abgezogen.)
# --------------------------------------------------
tax_rate = 0.2  # 20 % Steuer auf den Gewinn seit dem letzten Regimewechsel

# --- Simulation für 3x (klassisch) ---
portfolio_3x = 1.0
baseline_3x  = 1.0   # Setze Baseline beim Einstieg in leveraged Phase
last_signal_3x = spx.loc[0, 'signal_3x']
tax_portfolio_3x = []

# --- Simulation für 2x (klassisch) ---
portfolio_2x = 1.0
baseline_2x  = 1.0
last_signal_2x = spx.loc[0, 'signal_2x']
tax_portfolio_2x = []

# --- Simulation für 3x mit 200-Tage × 0.9 ---
portfolio_3x_0_9 = 1.0
baseline_3x_0_9  = 1.0
last_signal_3x_0_9 = spx.loc[0, 'signal_3x_0_9']
tax_portfolio_3x_0_9 = []

for i, row in spx.iterrows():
    daily_ret = row['daily_return']

    # --- 3x (klassisch) ---
    current_signal_3x = row['signal_3x']  # 3 oder 0
    if i == 0:
        tax_portfolio_3x.append(portfolio_3x)
    else:
        if current_signal_3x > 0:
            # Beim Wechsel von risk-off zu leveraged: Baseline neu setzen
            if last_signal_3x == 0:
                baseline_3x = portfolio_3x
            portfolio_3x = portfolio_3x * (1 + current_signal_3x * daily_ret)
        else:
            # risk-off: Wenn der Vortag noch leveraged war, wird der Gewinn seit der Baseline besteuert
            if last_signal_3x > 0:
                profit = portfolio_3x - baseline_3x
                if profit > 0:
                    portfolio_3x = portfolio_3x - tax_rate * profit
            # An risk-off-Tagen bleibt der Portfolio-Wert konstant
        tax_portfolio_3x.append(portfolio_3x)
        last_signal_3x = current_signal_3x

    # --- 2x (klassisch) ---
    current_signal_2x = row['signal_2x']  # 2 oder 0
    if i == 0:
        tax_portfolio_2x.append(portfolio_2x)
    else:
        if current_signal_2x > 0:
            if last_signal_2x == 0:
                baseline_2x = portfolio_2x
            portfolio_2x = portfolio_2x * (1 + current_signal_2x * daily_ret)
        else:
            if last_signal_2x > 0:
                profit = portfolio_2x - baseline_2x
                if profit > 0:
                    portfolio_2x = portfolio_2x - tax_rate * profit
        tax_portfolio_2x.append(portfolio_2x)
        last_signal_2x = current_signal_2x

    # --- 3x mit 200-Tage × 0.9 ---
    current_signal_3x_0_9 = row['signal_3x_0_9']  # 3 oder 0
    if i == 0:
        tax_portfolio_3x_0_9.append(portfolio_3x_0_9)
    else:
        if current_signal_3x_0_9 > 0:
            if last_signal_3x_0_9 == 0:
                baseline_3x_0_9 = portfolio_3x_0_9
            portfolio_3x_0_9 = portfolio_3x_0_9 * (1 + current_signal_3x_0_9 * daily_ret)
        else:
            if last_signal_3x_0_9 > 0:
                profit = portfolio_3x_0_9 - baseline_3x_0_9
                if profit > 0:
                    portfolio_3x_0_9 = portfolio_3x_0_9 - tax_rate * profit
        tax_portfolio_3x_0_9.append(portfolio_3x_0_9)
        last_signal_3x_0_9 = current_signal_3x_0_9

# Füge die steuerbereinigten Ergebnisse als neue Spalten ein
spx['strategy_cum_3x_tax']    = tax_portfolio_3x
spx['strategy_cum_2x_tax']    = tax_portfolio_2x
spx['strategy_cum_3x_tax_0_9'] = tax_portfolio_3x_0_9





In [375]:
spx.tail()

,Date,Close,daily_return,ma_200,in_risk_on,signal_3x,signal_2x,in_risk_on_0_9,signal_3x_0_9,strategy_return_3x,strategy_return_2x,strategy_return_3x_0_9,strategy_cum_3x,strategy_cum_2x,strategy_cum_3x_0_9,buy_and_hold,strategy_cum_3x_nontrading,strategy_cum_3x_tax,strategy_cum_2x_tax,strategy_cum_3x_tax_0_9
12623,2020-10-29,3310.110107,0.012117,3129.705349,True,3,2,True,3,0.036352,0.024235,0.036352,179908.007448,5946.937933,9.609698e+05,314.572472,280646.253327,15589.741874,931.076944,95074.802399
12624,2020-10-30,3269.959961,-0.011960,3129.471099,True,3,2,True,3,-0.035879,-0.023919,-0.035879,173453.151024,5804.692530,9.264915e+05,310.810327,270577.044642,15030.402982,908.806422,91663.646844
12625,2020-11-02,3310.239990,0.012488,3129.374198,True,3,2,True,3,0.037465,0.024976,0.037465,179951.505252,5949.672882,9.612022e+05,314.691789,280714.107426,15593.511131,931.505139,95097.789398
12626,2020-11-03,3369.159912,0.017969,3129.616047,True,3,2,True,3,0.053908,0.035939,0.053908,189652.307677,6163.495678,1.013019e+06,320.346577,295846.807151,16434.124108,964.982111,100224.308705
12627,2020-11-04,3443.439941,0.022217,3130.224497,True,3,2,True,3,0.066651,0.044434,0.066651,202292.850726,6437.365032,1.080537e+06,327.463733,315565.334954,17529.477262,1007.860218,106904.373420


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Annahme:
# Der DataFrame 'spx' enthält mindestens folgende Spalten:
#   Date         – Datum (als datetime oder String)
#   Close        – Schlusskurs
#   daily_return – Tagesrendite als Dezimalzahl (z.B. 0.01 für +1%)
# -------------------------------

# Beispiel: spx = pd.read_csv("deine_datei.csv")

# -------------------------------
# 1) Steuerfreie Berechnungen (ohne Steuer)
# -------------------------------

# 1.1) 200-Tage-Durchschnitt berechnen
spx['ma_200'] = spx['Close'].rolling(window=200).mean()

# 1.2) Klassischer Indikator: True, wenn Close über dem 200-Tage-SMA liegt
spx['in_risk_on'] = spx['Close'] > spx['ma_200']

# 1.3) Verzögertes Signal: Zustand des Vortages
spx['signal_3x'] = np.where(spx['in_risk_on'].shift(1, fill_value=True), 3, 0)
spx['signal_2x'] = np.where(spx['in_risk_on'].shift(1, fill_value=True), 2, 0)

# 1.4) Variante: Indikator mit 200-Tage × 0.9 (nur für 3x)
# Hier wird geprüft, ob der Close über 0.9×ma200 liegt.
spx['in_risk_on_0_9'] = spx['Close'] > (spx['ma_200'] * 0.98)
spx['signal_3x_0_9'] = np.where(spx['in_risk_on_0_9'].shift(1, fill_value=True), 3, 0)

# 1.5) Tagesrenditen (steuerfrei) der Strategien:
# Bei risk-on (Signal > 0) wird der jeweilige Hebelfaktor mit daily_return multipliziert.
# Bei risk-off (Signal == 0) gilt ab jetzt als Return 0.0001.
spx['strategy_return_3x']     = spx['signal_3x'] * spx['daily_return'] + (1 - (spx['signal_3x'] > 0).astype(int)) * 0.0001
spx['strategy_return_2x']     = spx['signal_2x'] * spx['daily_return'] + (1 - (spx['signal_2x'] > 0).astype(int)) * 0.0001
spx['strategy_return_3x_0_9'] = spx['signal_3x_0_9'] * spx['daily_return'] + (1 - (spx['signal_3x_0_9'] > 0).astype(int)) * 0.0001

# 1.6) Kumulative Performance (steuerfrei) – vektorisiert
spx['strategy_cum_3x']     = (1 + spx['strategy_return_3x']).cumprod()
spx['strategy_cum_2x']     = (1 + spx['strategy_return_2x']).cumprod()
spx['strategy_cum_3x_0_9'] = (1 + spx['strategy_return_3x_0_9']).cumprod()

# 1.7) Buy & Hold-Performance (ohne Hebel)
spx['buy_and_hold'] = (1 + spx['daily_return']).cumprod()

# 1.8) Nicht-tradendes, 3x-gehebeltes Portfolio: Hier wird täglich einfach der 3x-Faktor angewandt
spx['strategy_cum_3x_nontrading'] = (1 + 3 * spx['daily_return']).cumprod()

# 1.9) Entferne Zeilen, in denen ma_200 nicht berechnet werden konnte
spx = spx.dropna(subset=['ma_200']).reset_index(drop=True)

# -------------------------------
# 2) Simulation der Steuer-Anpassung (mit Steuer)
#    Beim Übergang vom Leveraged-Modus (Signal > 0) in risk-off (Signal == 0) wird 20 % des Gewinns
#    seit dem letzten Einstieg (Baseline) abgezogen.
#    Zusätzlich: An risk-off-Tagen wird nicht "0" sondern 0.0001 als Return genutzt.
# -------------------------------
tax_rate = 0.2  # 20 % Steuer

# --- Simulation für 3x (klassisch) ---
portfolio_3x = 1.0
baseline_3x  = 1.0
last_signal_3x = spx.loc[0, 'signal_3x']
tax_portfolio_3x = []

# --- Simulation für 2x (klassisch) ---
portfolio_2x = 1.0
baseline_2x  = 1.0
last_signal_2x = spx.loc[0, 'signal_2x']
tax_portfolio_2x = []

# --- Simulation für 3x mit 200-Tage × 0.9 ---
portfolio_3x_0_9 = 1.0
baseline_3x_0_9  = 1.0
last_signal_3x_0_9 = spx.loc[0, 'signal_3x_0_9']
tax_portfolio_3x_0_9 = []

# --- Simulation für 3x mit dualer Schwellenwertlogik (Upper: 0.98, Lower: 0.97), SHIFTED ---
# Hier wird das Signal anhand der Vortagesdaten bestimmt.
portfolio_dual = 1.0
initial_dual = 3 if spx.loc[0, 'Close'] >= (0.98 * spx.loc[0, 'ma_200']) else 0
last_signal_dual = initial_dual
flag_dual = False  # Flag, ob der untere Schwellenwert (0.97×ma200) berührt wurde
tax_portfolio_dual = []
# Zusätzlich führen wir für die Dual-Strategie ein "max_close_dual" ein, das das bisherige All-Time-High
# (seit dem letzten Entry in den Leveraged-Modus) speichert.
max_close_dual = spx.loc[0, 'Close'] if initial_dual > 0 else None

# Nun laufen wir über alle Zeilen:
for i in range(len(spx)):
    daily_ret = spx.iloc[i]['daily_return']

    # --- 2.A: 3x (klassisch) ---
    if i == 0:
        tax_portfolio_3x.append(portfolio_3x)
    else:
        current_signal_3x = spx.iloc[i]['signal_3x']  # 3 oder 0
        if current_signal_3x > 0:
            if last_signal_3x == 0:
                baseline_3x = portfolio_3x  # Re-Entry: Baseline setzen
            portfolio_3x *= (1 + current_signal_3x * daily_ret)
        else:
            if last_signal_3x > 0:
                profit = portfolio_3x - baseline_3x
                if profit > 0:
                    portfolio_3x -= tax_rate * profit
            # In risk-off: Statt 0-Rendite nutzen wir 0.0001
            portfolio_3x *= (1 + 0.0001)
        tax_portfolio_3x.append(portfolio_3x)
        last_signal_3x = current_signal_3x

    # --- 2.B: 2x (klassisch) ---
    if i == 0:
        tax_portfolio_2x.append(portfolio_2x)
    else:
        current_signal_2x = spx.iloc[i]['signal_2x']  # 2 oder 0
        if current_signal_2x > 0:
            if last_signal_2x == 0:
                baseline_2x = portfolio_2x
            portfolio_2x *= (1 + current_signal_2x * daily_ret)
        else:
            if last_signal_2x > 0:
                profit = portfolio_2x - baseline_2x
                if profit > 0:
                    portfolio_2x -= tax_rate * profit
            portfolio_2x *= (1 + 0.0001)
        tax_portfolio_2x.append(portfolio_2x)
        last_signal_2x = current_signal_2x

    # --- 2.C: 3x mit 200-Tage × 0.9 ---
    if i == 0:
        tax_portfolio_3x_0_9.append(portfolio_3x_0_9)
    else:
        current_signal_3x_0_9 = spx.iloc[i]['signal_3x_0_9']  # 3 oder 0
        if current_signal_3x_0_9 > 0:
            if last_signal_3x_0_9 == 0:
                baseline_3x_0_9 = portfolio_3x_0_9
            portfolio_3x_0_9 *= (1 + current_signal_3x_0_9 * daily_ret)
        else:
            if last_signal_3x_0_9 > 0:
                profit = portfolio_3x_0_9 - baseline_3x_0_9
                if profit > 0:
                    portfolio_3x_0_9 -= tax_rate * profit
            portfolio_3x_0_9 *= (1 + 0.0001)
        tax_portfolio_3x_0_9.append(portfolio_3x_0_9)
        last_signal_3x_0_9 = current_signal_3x_0_9

    # --- 2.D: Dual (3x) mit Schwellenwerten (Upper: 0.98, Lower: 0.97), SHIFTED ---
    # Bestimme das Signal anhand der Vortagesdaten:
    if i == 0:
        tax_portfolio_dual.append(portfolio_dual)
    else:
        prev_close = spx.iloc[i-1]['Close']
        prev_ma200 = spx.iloc[i-1]['ma_200']
        upper_val_prev = 0.98 * prev_ma200
        lower_val_prev = 0.97 * prev_ma200
        if prev_close >= upper_val_prev:
            current_signal_dual = 3
            flag_dual = False  # Flag zurücksetzen, da oberhalb der oberen Grenze
        elif prev_close <= lower_val_prev:
            current_signal_dual = 0
            flag_dual = True   # untere Grenze berührt
        else:
            current_signal_dual = 3 if flag_dual else 0

        # In der dualen Strategie: Wenn risk-off (Signal 0) gilt,
        # soll normalerweise ein Return von 0.0001 genutzt werden.
        # Zusätzlich wird geprüft: Hat der Close-Preis seit dem letzten Re-Entry (während der Leveraged-Phase)
        # ein All-Time-High erreicht, und ist der aktuelle Close mehr als 35 % darunter?
        # Falls ja, wird statt 0.0001 der unhebelte daily_return genutzt.
        if current_signal_dual > 0:
            if last_signal_dual == 0:
                baseline_dual = portfolio_dual
                # Re-Entry: Setze max_close_dual neu auf den aktuellen Close
                max_close_dual = spx.iloc[i]['Close']
            portfolio_dual *= (1 + current_signal_dual * daily_ret)
            # Aktualisiere max_close_dual
            if max_close_dual is not None:
                max_close_dual = max(max_close_dual, spx.iloc[i]['Close'])
        else:
            if last_signal_dual > 0:
                profit = portfolio_dual - baseline_dual
                if profit > 0:
                    portfolio_dual -= tax_rate * profit
            # Bestimme effektiven risk-off Return: Standardmäßig 0.0001...
            effective_risk_off = 0.0001
            # ...aber falls der aktuelle Close mehr als 35 % unter max_close_dual liegt, dann nutze daily_return.
            if max_close_dual is not None:
                drop_ratio = (max_close_dual - spx.iloc[i]['Close']) / max_close_dual
                if drop_ratio > 0.55:
                    effective_risk_off = daily_ret
            portfolio_dual *= (1 + effective_risk_off)
        tax_portfolio_dual.append(portfolio_dual)
        last_signal_dual = current_signal_dual

# Füge die steuerbereinigten Ergebnisse als neue Spalten ein
spx['strategy_cum_3x_tax']    = tax_portfolio_3x
spx['strategy_cum_2x_tax']    = tax_portfolio_2x
spx['strategy_cum_3x_tax_0_9'] = tax_portfolio_3x_0_9
spx['strategy_cum_3x_tax_dual'] = tax_portfolio_dual

# -------------------------------
# 3) Ausgabe einiger Spalten zum Vergleich
# -------------------------------
spx.tail()



NameError: name 'baseline_dual' is not defined

In [114]:

spx["base_portfolio"] = 100 * (1 + spx["daily_return"]).cumprod()
spx["200SMA"] = SPX_PRICE["Close"].rolling(200).mean()
spx["adj_daily_return"] = 0


In [115]:
spx["adj_daily_return"] = 0.0
spx["adj_daily_return"] = spx["daily_return"].shift(1)
spx["adj_daily_return"] = spx["adj_daily_return"].where(spx["Close"].shift(2) >= spx["200SMA"].shift(2), 0)

In [116]:
spx["adj_daily_return_3x"]=spx["adj_daily_return"]*3

In [117]:
spx["lev_port_3x_base"] = 100
spx["adj_portfolio_no_lev"] = 100 * (1 + spx["adj_daily_return"]).cumprod()
spx["lev_port_3x_base"] = 100 * (1 + spx["adj_daily_return_3x"]).cumprod()

In [118]:
spx.head()

,Date,Close,daily_return,base_portfolio,200SMA,adj_daily_return,adj_daily_return_3x,lev_port_3x_base,adj_portfolio_no_lev
200,1928-10-17,21.790001,NaN,NaN,19.20440,0.000000,0.000000,100.000000,100.000000
201,1928-10-18,21.840000,0.002435,100.243458,19.22480,0.000000,0.000000,100.000000,100.000000
202,1928-10-19,21.959999,0.005634,100.808276,19.24600,0.002435,0.007304,100.730375,100.243458
203,1928-10-22,21.809999,-0.006691,100.133808,19.26730,0.005634,0.016903,102.433059,100.808276
204,1928-10-23,21.950001,0.006559,100.790602,19.28875,-0.006691,-0.020072,100.377043,100.133808


In [119]:
spx.tail()

,Date,Close,daily_return,base_portfolio,200SMA,adj_daily_return,adj_daily_return_3x,lev_port_3x_base,adj_portfolio_no_lev
23318,2020-10-29,3310.110107,0.012087,386265.887125,3129.705349,-0.035148,-0.105444,3.143136e+10,272798.127921
23319,2020-10-30,3269.959961,-0.011990,381634.732963,3129.471099,0.012087,0.036262,3.257112e+10,276095.529151
23320,2020-11-02,3310.239990,0.012458,386389.215751,3129.374198,-0.011990,-0.035969,3.139958e+10,272785.268003
23321,2020-11-03,3369.159912,0.017939,393320.763998,3129.616047,0.012458,0.037375,3.257313e+10,276183.682114
23322,2020-11-04,3443.439941,0.022187,402047.390517,3130.224497,0.017939,0.053818,3.432615e+10,281138.221318
